In [1]:
import os
from pathlib import Path

import numpy as np
import optuna
import pandas as pd
import torch

from mscproject.datasets import CompanyBeneficialOwners
import mscproject.models as mod

while not Path("data") in Path(".").iterdir():
    os.chdir("..")

In [2]:
study = optuna.create_study(
    direction="minimize",
    study_name="pyg_model_selection",
    storage="sqlite:///pyg_model_selection.db",
    load_if_exists=True,
)

# optimise_model_partial = ft.partial(optimise_model, dataset=dataset, device=device)
# study.optimize(optimise_model_partial, n_trials=0)

[I 2022-09-05 22:12:25,893] Using an existing study with name 'pyg_model_selection' instead of creating a new one.


In [23]:
# Print top models.
drop_cols = ["number", "datetime_start", "datetime_complete", "duration", "state"]
trials_df: pd.DataFrame = study.trials_dataframe()
trials_df = trials_df.drop(drop_cols, axis=1, errors="ignore")
trials_df = trials_df.sort_values("value")

In [24]:
print("Top models")
top = trials_df.head(10).T
print(top)

Top models
                                  163       101       112       181       162  \
value                        1.062521  1.064484  1.064862   1.06536  1.067316   
params_act                       gelu      gelu      gelu      gelu      gelu   
params_act_first                 True      True      True      True      True   
params_bias                      True      True      True      True      True   
params_concat                     NaN       NaN       NaN       NaN       NaN   
params_dropout               0.218731  0.224939  0.244093  0.245872  0.242771   
params_edge_aggr                  min       min       min       min       min   
params_gcn_aggr                   max       max       max       max       max   
params_heads                      NaN       NaN       NaN       NaN       NaN   
params_hidden_channels_log2       6.0       6.0       6.0       6.0       6.0   
params_jk                        none      none      none      none      none   
params_model     

In [25]:
trials_df.describe().T

,count,mean,std,min,25%,50%,75%,max
value,201.0,1.139121,6.249137e-02,1.062521,1.085543,1.119437,1.185594,1.250485
params_dropout,205.0,0.217525,9.931742e-02,0.000290,0.168151,0.231134,0.267497,0.499698
params_heads,14.0,6.071429,5.837074e+00,1.000000,2.000000,4.000000,8.000000,16.000000
params_hidden_channels_log2,205.0,5.741463,1.399050e+00,0.000000,6.000000,6.000000,6.000000,8.000000
params_n_layers,205.0,2.800000,6.055301e-01,1.000000,3.000000,3.000000,3.000000,4.000000
params_weight_decay,205.0,0.013776,2.176766e-02,0.000013,0.002098,0.004953,0.011835,0.099774
user_attrs_best_epoch,197.0,54.126904,3.047965e+01,2.000000,36.000000,54.000000,73.000000,177.000000
user_attrs_learning_rate,205.0,0.010000,1.738970e-18,0.010000,0.010000,0.010000,0.010000,0.010000
user_attrs_n_hidden,205.0,74.263415,5.751947e+01,1.000000,64.000000,64.000000,64.000000,256.000000
user_attrs_total_epochs,201.0,63.069652,3.107660e+01,11.000000,43.000000,64.000000,82.000000,187.000000


In [4]:
# Plot the results.
optuna.visualization.plot_optimization_history(study).show()
optuna.visualization.plot_contour(study).show()
optuna.visualization.plot_slice(study).show()
optuna.visualization.plot_param_importances(study).show()

In [5]:
best = study.best_params

In [6]:
best

{'act': 'gelu',
 'act_first': True,
 'bias': True,
 'dropout': 0.2187307625980795,
 'edge_aggr': 'min',
 'gcn_aggr': 'max',
 'hidden_channels_log2': 6,
 'jk': 'none',
 'model': 'GCN',
 'n_layers': 3,
 'weight_decay': 0.0016682697954684728}

In [7]:
dataset_path = "data/pyg/"

# Set the device.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the dataset.
dataset = CompanyBeneficialOwners(dataset_path, to_undirected=True)
dataset = dataset.data.to(device)

In [8]:
model = mod.get_model("HAN")

In [9]:
model(in_channels=-1, hidden_channels=64, num_layers=3, metadata=dataset.metadata())

HAN(-1, 64, num_layers=3)

In [11]:
dataset

HeteroData(
  company={
    x=[96530, 32],
    y=[96530],
    train_mask=[96530],
    val_mask=[96530],
    test_mask=[96530],
    feature_names=[32]
  },
  person={
    x=[32609, 18],
    y=[32609],
    train_mask=[32609],
    val_mask=[32609],
    test_mask=[32609],
    feature_names=[18]
  },
  (company, owns, company)={
    edge_index=[2, 54607],
    edge_attr=[54607, 1],
    train_mask=[54607],
    val_mask=[54607],
    test_mask=[54607]
  },
  (person, owns, company)={
    edge_index=[2, 80219],
    edge_attr=[80219, 1],
    train_mask=[80219],
    val_mask=[80219],
    test_mask=[80219]
  },
  (company, rev_owns, company)={
    edge_index=[2, 54607],
    edge_attr=[54607, 1],
    train_mask=[54607],
    val_mask=[54607],
    test_mask=[54607]
  },
  (company, rev_owns, person)={
    edge_index=[2, 80219],
    edge_attr=[80219, 1],
    train_mask=[80219],
    val_mask=[80219],
    test_mask=[80219]
  }
)